In [3]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# driver.implicitly_wait(5)
# s_data = []

# driver.get('https://sg.sbiz.or.kr/godo/stat/upso.sg')
# driver.find_element(By.CSS_SELECTOR,'a#twoTypePopup').click()
# time.sleep(1)
# si = bizinfo_df['address'][0][:2]
# driver.find_element(By.XPATH, f"//label[text()='{si}']").click()   #(By.XPATH, "//label[text()='"+si+"']").click()

In [55]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)
time.sleep(0.5)
s_data, 행정구역, 행정구역, 업종, 주간인구수, 밀집도, 밀집도_중위값, 밀집정도, 동종근로자_임금_수준미만, 동종근로자_임금_수준미만_척도 = [],[],[],[],[],[],[],[],[],[] 

driver.get('https://sg.sbiz.or.kr/godo/stat/updens.sg')
for region in range(1,18):
    for industry in range(2,23):
        driver.implicitly_wait(10)
        time.sleep(0.5)
        driver.find_element(By.XPATH,'//*[@id="page4-1"]/div[1]/div/div[1]/ul/li[1]/div').click()
        driver.implicitly_wait(10)
        time.sleep(0.5)
        for i in range(1, region+1):
            driver.find_element(By.XPATH,f'//*[@id="megaUl"]/li[{i+1}]/label').click()
            driver.implicitly_wait(10)
            time.sleep(0.5)
            if i % 5 == 0 :
                driver.find_element(By.TAG_NAME, 'html').send_keys(Keys.ARROW_DOWN)
        driver.implicitly_wait(10)
        time.sleep(0.5)
        driver.find_element(By.XPATH, '//*[@id="ctyUl"]/li[1]/label').click()
        driver.implicitly_wait(10)
        time.sleep(0.5)
        driver.find_element(By.XPATH, '//*[@id="popupbox"]/div/div[2]/div/div[2]/a[2]').click()  
        driver.implicitly_wait(10)
        time.sleep(0.5)
        driver.find_element(By.XPATH,'//*[@id="page4-1"]/div[1]/div/div[1]/ul/li[2]/div').click()
        driver.implicitly_wait(10)
        time.sleep(0.5)
        for nm in range(1, industry+1):
            driver.implicitly_wait(10)
            time.sleep(0.5)
            driver.find_element(By.XPATH,f'//*[@id="upjong1Ul"]/li[{nm}]/label').click()
            driver.implicitly_wait(10)
            time.sleep(0.5)
            if nm % 3 == 0 :
                driver.find_element(By.TAG_NAME, 'html').send_keys(Keys.ARROW_DOWN)
            if nm % 15 == 0 :
                driver.find_element(By.TAG_NAME, 'html').send_keys(Keys.ARROW_DOWN)
        driver.implicitly_wait(10)
        time.sleep(0.5)
        driver.find_element(By.XPATH, '//*[@id="upjong2Ul"]/li[1]/label').click()
        driver.implicitly_wait(10)
        time.sleep(0.5)
        driver.find_element(By.XPATH, '//*[@id="popupbox"]/div/div[2]/div/div[2]/a[2]').click()
        driver.implicitly_wait(10)
        time.sleep(0.5)
        driver.find_element(By.XPATH, '//*[@id="page4-1"]/div[1]/div/div[2]/a').click()
        driver.implicitly_wait(10)
        time.sleep(0.5)
        page_source = driver.page_source
        soup = bs(page_source, 'lxml')
        try:
            soup.select('#updensTable tr')[1]
        except: pass
        else:
            table = soup.select('#updensTable td')
            for i in range(len(table)//9):
                행정구역 = [table[9*i].text]
                업종 = [table[9*i+1].text]
                주간인구수 = [table[9*i+2].text]
                밀집도 = [table[9*i+3].text]
                밀집도_중위값 = [table[9*i+4].text]
                밀집정도 = [table[9*i+5].text]
                동종근로자_임금_수준미만 = [table[9*i+6].text]
                동종근로자_임금_수준미만_척도 = [table[9*i+7].text]
                s_data.append([행정구역, 업종, 주간인구수, 밀집도, 밀집도_중위값, 밀집정도, 동종근로자_임금_수준미만, 동종근로자_임금_수준미만_척도])

ElementClickInterceptedException: Message: element click intercepted: Element <div data-skin="selectbox-group">...</div> is not clickable at point (553, 340). Other element would receive the click: <div style="position:fixed; left:50%; top:50%; width:30em; height:30em;-webkit-transform:translate(-50%, -50%);transform:translate(-50%, -50%);">...</div>
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x002F1ED3+2236115]
	Ordinal0 [0x002892F1+1807089]
	Ordinal0 [0x001966FD+812797]
	Ordinal0 [0x001CBEC7+1031879]
	Ordinal0 [0x001C9E6C+1023596]
	Ordinal0 [0x001C7A5B+1014363]
	Ordinal0 [0x001C66E7+1009383]
	Ordinal0 [0x001BC416+967702]
	Ordinal0 [0x001E1A8C+1120908]
	Ordinal0 [0x001BBD84+966020]
	Ordinal0 [0x001E1CA4+1121444]
	Ordinal0 [0x001F59E2+1202658]
	Ordinal0 [0x001E18A6+1120422]
	Ordinal0 [0x001BA73D+960317]
	Ordinal0 [0x001BB71F+964383]
	GetHandleVerifier [0x0059E7E2+2743074]
	GetHandleVerifier [0x005908D4+2685972]
	GetHandleVerifier [0x00382BAA+532202]
	GetH

In [ ]:
sbiz_df = pd.DataFrame(s_data, columns=['행정구역', '업종', '주간인구수', '밀집도', '밀집도_중위값', '밀집정도', '동종근로자_임금_수준미만', '동종근로자_임금_수준미만_척도'])
sbiz_df.to_csv('sbiz_df.csv') 